In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import model_selection

from sklearn.metrics import mean_squared_error
from optuna.integration import lightgbm as lgb

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Blend with LightGBM

In [2]:
# Load the training data
df = pd.read_csv("../input/30-days-create-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

In [3]:
df1 = pd.read_csv("../input/blending-blending-blending/train_pred_1.csv")
df2 = pd.read_csv("../input/blending-blending-blending/train_pred_2.csv")
df3 = pd.read_csv("../input/blending-blending-blending/train_pred_3.csv")

df_test1 = pd.read_csv("../input/blending-blending-blending/test_pred_1.csv")
df_test2 = pd.read_csv("../input/blending-blending-blending/test_pred_2.csv")
df_test3 = pd.read_csv("../input/blending-blending-blending/test_pred_3.csv")

df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")

df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont9,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3
0,149020,A,B,B,C,B,C,A,E,E,...,0.292799,0.130800,0.295031,0.800303,0.869909,7.887014,0,8.100012,8.072755,8.112833
1,477059,A,B,B,C,B,B,A,E,C,...,0.635890,0.328430,0.322531,0.021058,0.153536,7.152708,0,8.373678,8.439766,8.384217
2,104376,A,A,A,C,B,B,A,E,E,...,0.322940,0.450631,0.246561,0.236413,0.822472,7.869662,0,8.236361,8.206692,8.199950
3,488682,B,A,A,C,B,B,A,E,C,...,0.467147,0.292676,0.734973,0.351734,0.481341,8.112649,0,8.327992,8.361216,8.368644
4,281982,A,B,A,A,B,D,A,E,A,...,0.579008,0.648271,0.590032,0.296360,0.287297,8.186170,0,7.891100,7.828485,7.872571


In [4]:
df_test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont7,cont8,cont9,cont10,cont11,cont12,cont13,pred_1,pred_2,pred_3
0,0,B,B,B,C,B,B,A,E,E,...,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702,8.069092,8.063132,8.085993
1,5,A,B,A,C,B,C,A,E,C,...,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940,8.399485,8.393074,8.411225
2,15,B,A,A,A,B,B,A,E,D,...,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099,8.402053,8.414242,8.418512
3,16,B,B,A,C,B,D,A,E,A,...,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372,8.497690,8.505300,8.520524
4,17,B,B,A,C,B,C,A,E,C,...,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412,8.119550,8.124400,8.105719


In [5]:
# LGBMのパラメータを設定
param = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "fair",
    "verbosity": -1,
    "seed": 42
}

In [6]:
useful_features = ["pred_1", "pred_2", "pred_3"]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(10):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    lgb_train = lgb.Dataset(xtrain, ytrain)
    lgb_valid = lgb.Dataset(xvalid, yvalid)

    model = lgb.train(param,
                      lgb_train,
                      valid_names=['train', 'valid'],
                      valid_sets=[lgb_train, lgb_valid],
                      num_boost_round=10000,
                      early_stopping_rounds=300,
                      verbose_eval=500)
    
    preds_valid = model.predict(xvalid,num_iteration=model.best_iteration)
    test_preds = model.predict(xtest,num_iteration=model.best_iteration)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

[I 2021-08-31 15:03:12,902] A new study created in memory with name: no-name-817d761b-a684-4cdc-9fe4-cb7c5a4b4831
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 300 rounds


feature_fraction, val_score: 0.153692:  14%|#4        | 1/7 [00:04<00:26,  4.36s/it]

Early stopping, best iteration is:
[37]	train's fair: 0.152872	valid's fair: 0.153692
Training until validation scores don't improve for 300 rounds


feature_fraction, val_score: 0.153692:  29%|##8       | 2/7 [00:08<00:21,  4.28s/it]

...

min_data_in_leaf, val_score: 0.152818:  80%|########  | 4/5 [00:22<00:05,  5.42s/it]

[500]	train's fair: 0.153133	valid's fair: 0.152865
Early stopping, best iteration is:
[210]	train's fair: 0.153173	valid's fair: 0.152827
Training until validation scores don't improve for 300 rounds


min_data_in_leaf, val_score: 0.152818: 100%|##########| 5/5 [00:27<00:00,  5.53s/it]

[500]	train's fair: 0.153131	valid's fair: 0.152854
Early stopping, best iteration is:
[210]	train's fair: 0.153176	valid's fair: 0.152823


9 0.7150882436501484
0.7157462313443468 0.0010793748780640441


In [7]:
sample_submission.target = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.to_csv("0831_blend_submission.csv", index=False)